
# Table of Contents
   1. pandas加载数据                                                                                 
      1.1 熟悉数据集                                                                              
      1.2 sklearn拆分数据集                                                                           
      1.3 pandas df转为pyspark df                                                                       
      1.3.1 熟悉sql.df的属性和方法                                                                       
   2. 构建ALS模型                                                                                   
   3. 为用户推荐评分Top-5的电影                                                                           
       3.1 生成为所有训练集用户推荐的结果                                                                
       3.2 为指定的用户和电影预测评分                                                                     
   4. 评估模型                                                                                      
   5. ALS参数不同取值，得到的均方根误差值汇总表如下：                                                          
   6. 总结：
   
    ###  Action01项目要求：                                                                           
           对MovieLens数据集进行评分预测
           工具：可以使用Surprise或者其他                                                                 
           说明使用的模型，及简要原理                                                                   
           我们需要补全评分矩阵，然后对指定用户，比如userID为1-5进行预测                                           
   #### jupyter中使用pyspark，需要先通过findspark运行。                                                     

In [3]:
import findspark
findspark.__version__

'1.4.2'

In [2]:
!pip install findspark

In [1]:
# jupyter连接pyspark
findspark.init()

NameError: name 'findspark' is not defined

In [1]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'

In [2]:
!pip install pyspark

^C


In [8]:
pyspark.__version__

NameError: name 'pyspark' is not defined

In [ ]:
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator # 回归器评估

In [ ]:
from pyspark.sql.types import * # 导入它的目的不清楚

In [ ]:
from sklearn.model_selection import train_test_split # 拆分数据集

## 1. pandas加载数据

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
#用pandas读取
df = pd.read_csv('ratings.csv',
                usecols=['userId','movieId','rating'])
df.head(2)

### 1.1 熟悉数据集

In [ ]:
df.describe()

In [ ]:
userid = df['userId'].unique()
movieid = df['movieId'].unique()

In [ ]:
userid.shape,movieid.shape

In [ ]:
# 统计每部电影共被打分过多少次，默认按value降序
temp = df['movieId'].value_counts()

In [ ]:
temp[temp == 1]

In [ ]:
new_df = df.set_index(keys='movieId',drop=False)
new_df.head(2)

In [ ]:
# 按被打分次数由高到低重新提取数据集
new_df = new_df.loc[temp.index]

In [ ]:
# 只被评分过一次的电影
singles = new_df.iloc[-2558:]
singles.head(2)

In [ ]:
# 被评分过2次及以上的电影
need_spilted = new_df.iloc[:-2558]

### 1.2 sklearn拆分数据集

#### pyspark的sqldf.randomSplit()方法拆分数据集时，无法实现分层抽样。为了保证每个userid都可以被拆分到2个数据集中，所用sklearn的train_test_split进行拆分；

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(need_splited[['userId','movieId']],
                                                need_splited['rating'],
                                                test_size=0.2,
                                                random_state=30,
                                                stratify=need_spilted['movieId'])
type(x_train),type(y_train)

In [ ]:
# 拼接被拆分的label和特征
x_train['rating'] = y_train
x_test['rating'] = y_test
x_train.shape, x_test.shape

In [ ]:
x_train['userId'].unique().shape,x_test['userId'].unique().shape

In [ ]:
x_train['movieId'].unique().shape,x_test['movieId'].unique().shape

In [ ]:
# 把被打分过一次的电影拼接进训练集中
x_train_ = pd.concat((x_train,singles),axis=0)

In [ ]:
# 原数据集中用户id数和电影id数
userid.shape, movieid.shape

In [ ]:
x_train_['userId'].unique().shape,x_train_['movieId'].unique().shape


#### 拆分后训练集用户id数与总数据集用户id数相同，训练集的电影id与总数据集用户id数相同，说明训练集中已包含所有不同用户id和所有不同电影id。测试集用户id数、电影id数均少于训练集。但训练集中已包含所有用户id和电影id，因此可以确保测试集评估时不再出现空值。

## 1.3 pandas df 转为pyspark df

In [ ]:
sc = SparkContext()

In [ ]:
sql_sc = SQLContext(sc)

In [ ]:
%%time
# 把pandas的DataFrame转为pyspark下的DataFrame
spk_df_train = sql_sc.createDataFrame(data=x_train_)
spk_df_test = sql_sc.createDataFrame(data=x_test)



### 1.3.1 熟悉sql.df的属性和方法

In [ ]:
# 也是DataFrame类型，继承了pandas中部分df的方法
type(spk_df_train)

In [ ]:
# 查看数据列信息
spk_df_train.printSchema()

In [ ]:
# 查看总行数和字段列表
spk_df_train.count(), spk_df_train.columns

In [ ]:
## 将每条记录转成Row对象，以列表形式返回
# temp = spark_df_ratings.collect()

In [ ]:
# 查看训练集前5条记录, 输出结果为sql数据表风格
spk_df_train.show(n=5,truncate=True,vertical=False)

In [ ]:
# 查看测试集前5条，take方法以列表形式返回Row对象
spk_df_test.take(5)

In [ ]:
%%time
#支持where条件语句查询
spk_df_train.where(spk_df_train.userId==1).take(2)

# 2 构建ALS模型
ALS显示反馈目标函数：$$\underset{x,y}{min}\sum _{(u, i)\varepsilon K}(r_{ui} - x_{u}^{T}y_{i})^2 + \lambda (\sum _{u}||x_{u}||_{2}^{2} + \sum _{i}||y_{i}||_{2}^{2})$$

$r_{ui}$实际评分                                                                                    
$x_{u}^{T}y_{i}$用户向量与物品向量的内积，表示用户u 对物品i 的预测评分                                                
$\lambda (\sum _{u}||x_{u}||_{2}^{2} + \sum _{i}||y_{i}||_{2}^{2})$ L2正则项，保证数值计算稳定性，防止过拟合                      
pyspark的API:                                                                                      

    pyspark.ml.recommendation.ALS()                                                                           
参数：                                                                                          
     rank=10, 模型中潜在因子的数量（默认10，一般取10~1000，太小误差大；太大泛化能力差）                                         
     maxIter=10, 要运行的最大迭代次数（ 默认为 10）                                                                 
     regParam=0.1, 指定 ALS 中的正则化参数（ 默认为 0.1）                                                             
numUserBlocks=10, 用户矩阵将被分区为块以便并行计算的块数（默认为10）                                                
numItemBlocks=10, 项目矩阵将被分区为块以便并行计算的块数（默认为10)                                                   
implicitPrefs=False, 指定是使用显式反馈ALS变体，还是使用适用于隐式反馈数据的变体；默认值 False，表示使用显式反馈                     
alpha=1.0, 适用于 ALS 的隐式反馈变量的参数，其控制偏好观察中的基线置信度（ 默认为 1.0）                                    
userCol='user', 用户列的字段名称，默认'user'                                                                
itemCol='item', 项目列的字段名称，默认'item'                                                                
seed=None,                                                                                         
ratingCol='rating', 评分列的字段名称，默认'rating'                                                            
nonnegative=False, 指定是否对最小二乘使用非负约束（ 默认False）                                                    
checkpointInterval=10, 每10次迭代检查一次                                                                  
intermediateStorageLevel='MEMORY_AND_DISK',                                                                   
finalStorageLevel='MEMORY_AND_DISK',                                                                      
coldStartStrategy='nan',                                                                              
blockSize=4096                                                                                    

In [ ]:
# 实例化模型
als = ALS(
    rank=3,  
    maxIter=10,
    regParam=0.1,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating'
)

In [ ]:
%%time
# 训练模型
model = als.fit(spk_df_train)

# 3 为用户推荐评分Top-5的电影

## 3.1 生成为所有训练集用户推荐的结果

In [ ]:
%%time
# 为所有训练集用户推荐Top-5
recommends = model.recommendForAllUsers(numItems=5)

In [ ]:
type(recommends)

In [ ]:
%%time
# 查找模型生成的为userID=100推荐的top5电影明细
user100 = recommends.where(recommends['userId'] == 100)
user100.collect()

In [ ]:
type(user100)

In [ ]:
user100.printSchema()

In [ ]:
df[df['userId'] == 100][(df['movieId'] == 95218) | \
   (df['movieId'] == 40697) | (df['movieId'] == 4261) | (df['movieId'] ==82931)]

#### 查询没结果，说明推荐的top-5电影都是用户以前未评分过的，ALS模型填充了评分矩阵中缺失的评分，使得这些评分能反应用户的喜好程度，从而让我们能利用这个预测评分排名后，选取了top-5部电影给指定用户。


## 3.2 为指定的用户和电影预测评分

In [ ]:
test = sql_sc.createDataFrame([[100, 95218], 
                               [100, 40697],
                               [100, 4261],
                               [100, 82931],
                               [100, 95776]], 
                               ['userId', 'movieId'])

In [ ]:
# 为指定用户和电影预测评分，按评分排序
predictions = sorted(model.transform(test).collect(), 
                     key=lambda r: r[2], reverse=True)
predictions


#### 可以看到预测的评分值，与3.1中recommends=model.recommendForAllUsers()中的对应用户和电影的评分一致。不同的是，recommends中只保存了每个用户预测评分top5的电影；而model.transform()可以为任何指定的用户id和电影id预测其评分。

# 4评估模型

In [ ]:
df.head()

In [ ]:
# 取df中前5条数据进行预测, 按movieId升序排序
test = sql_sc.createDataFrame(df.iloc[:5])

sorted(model.transform(test).collect(), 
       key=lambda r: r[1], reverse=False)

#### 可以看出预测值和实际值之间有出入，引入回归评估器评估预测效果：

In [ ]:
preds = model.transform(spk_df_test)
preds.columns

In [ ]:
preds.take(3)

In [ ]:
# 查看数据列信息
preds.printSchema()

In [ ]:
%%time
# 测试集记录数209204，非空值209204，说明出现0条空值记录
preds.dropna().count()

In [ ]:
# 构建回归评估器，评估误差
evaluator = RegressionEvaluator(metricName='rmse', 
                                predictionCol='prediction',
                                labelCol='rating')

In [ ]:
%%time
rms = evaluator.evaluate(preds.dropna())

In [ ]:
#  # 0.8395
"均方根误差值: {:.4f}".format(rms)

## 5 ALS参数不同取值，得到的均方根误差值汇总表如下：
序号	rank	maxIter	regParam	rmse
1	3	10	0.01	0.8528
2	3	10	0.1	0.8395
3	3	15	0.1	0.8370
4	3	20	0.1	0.8376
5	3	10	0.5	0.9983
6	10	10	0.5	0.9982
7	10	10	0.1	0.8239
8	20	10	0.1	0.8219
9	20	20	0.1	0.8162
10	50	20	0.1	0.8153
11	3	10	2	2.1745


In [ ]:
0.8153 / 5.0

# 6 总结：
    关于ALS算法：
它通过userCol指定的用户列, 和itemCol指定的项目列，由implicitPrefs指定处理的是显示反馈还是隐式行为的场景，根据rank指定的隐特征个数拟合评分矩阵；拟合后再通过recommendForAllUsers(N)为所有用户生成评分最高的Top-N的推荐结果，根据指定的用户id即可在生成的结果中查找推荐结果。

pyspark进行拟合时，ALS的rank, maxIter, regParam三个参数的取值会影响rmse。

当rank太小时，单纯提高迭代次数maxIter很快会过拟合。

汇总表序号2和序号10的结果可以看出，rank、maxIter同时提高取值，rmse会更好的降低；但增大rank和maxIter耗时会增加，rank取值1000的时候，跑了1个多小时仍没跑完。

    其他注意事项：
        1.必须保证训练集中包含所有的用户id和电影id，即：要保证训练集样本覆盖全部用户和电影，如：该数据集中共有7120个不同用户、14026部不同电影，则训练集中必须包含7120个用户和14026部电影下的部分数据；
        2.如果训练集中未覆盖全部用户和电影，则训练集评估时预测值将会出现空值，导致rmse也是空值，影响拟合；
        3.虽然真实数据集中rating的值在[0.5, 5.0]范围，而预测评分会出现大于5.0的情况。经过多次尝试，发现当rmse误差值降低后，大于5.0的评分情况会减少，所以猜测是因为误差过大导致（最低rmse占最高真实评分5.0的16.31%）；
        4.预测结果DataFrame的字段prediction数据列信息是: float (nullable = false)，当nullable = false不代表一定原空值;
        5.SQLContext是Spark SQL进行结构化数据处理的入口，它兼具了pandas的DataFrame的大部分方法和SQL语句，实际应用中可以多尝试。